<a href="https://colab.research.google.com/github/emisoft-designs/Data-Science-Practice-Projects/blob/main/NN_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import seed
from random import random
from random import randrange
from math import exp
from csv import reader
import pandas as pd
import numpy as np

In [ ]:
class neural_net():
  def __init__(self, n_inputs, n_hidden, n_outputs):
    self.network = list() 
    hidden_layers = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    self.network.append(hidden_layers)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    self.network.append(output_layer)

  def activate(self, weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
      activation += weights[i] * inputs[i]
    return activation

  def transfer(self, x, activation='relu'):
    if activation == 'sigmoid':
      return 1.0/(1.0 + exp(-x))
    if activation == 'relu':
      return max(0,x)
    
  def transfer_derivative(self, x, activation='relu'):
    if activation == 'sigmoid':
      return x * (1.0 - x)
    if activation == 'relu':
      return (1 if x>0 else 0)

  # Make a prediction with a network
  def predict(self,row):
	  outputs = self.forward_propagate(row)
	  return outputs.index(max(outputs))
   
  def forward_propagate(self, input_vals):
    inputs = input_vals
    #
    for layer in self.network:
      new_inputs = []
      for neuron in layer:
        activation = self.activate(neuron['weights'], inputs)
        neuron['output'] = self.transfer(activation)
        new_inputs.append(neuron['output'])
      inputs = new_inputs
    return inputs

  def backward_propagate_error(self, expected):
    for i in reversed(range(len(self.network))):
      layer = self.network[i]
      errors = list()
      if i != len(self.network)-1:
        for j in range(len(layer)):
          error = 0.0
          for neuron in self.network[i + 1]:
            error += (neuron['weights'][j] * neuron['delta'])
            errors.append(error)
      else:
        for j in range(len(layer)):
          neuron = layer[j]
          errors.append(neuron['output'] - expected[j])
      for j in range(len(layer)):
        neuron = layer[j]
        neuron['delta'] = errors[j] * self.transfer_derivative(neuron['output'])
       
  
  def update_weights(self, row, l_rate):
    for i in range(len(self.network)):
      inputs = row[:-1]
      if i != 0:
        inputs = [neuron['output'] for neuron in self.network[i - 1]]
      for neuron in self.network[i]:
        for j in range(len(inputs)):
          neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
        neuron['weights'][-1] -= l_rate * neuron['delta']
  
  def train(self, dataset, target_set, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
      sum_error = 0
      for row in dataset:
        outputs = self.forward_propagate(row)
        expected  =  target_set

        if(len(outputs) != len(expected)):
          print("train set out of shape")
          break

        sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
        self.backward_propagate_error(expected)
        self.update_weights(row, l_rate)
      print('>epoch=%d, lrate=%.3f, error=%.3f, accurace=%.3f' % (epoch, l_rate, sum_error,(1-sum_error)*100))
  

In [ ]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [ ]:
def clean_string(x):
  num_list = x[0].replace("\t",',')
  num_list = num_list.split(',')
  float_list = [float(x) for x in num_list if x != '']
  return float_list

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return (correct / float(len(actual)) )* 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
# Backpropagation Algorithm With Stochastic Gradient Descent
def fit(train, test, l_rate, n_epoch, n_hidden):
  n_inputs = len(train[0]) - 1
  n_outputs = len(set([row[-1] for row in train]))
  network = neural_net(n_inputs, n_hidden, n_outputs)
  for row in train:
    expected = [0 for i in range(n_outputs)]
    expected[int(row[-1])] = 1
  network.train(train, expected,l_rate, n_epoch, n_outputs)
  predictions = list()
  for row in test:
    prediction = network.predict(row)
    predictions.append(prediction)
  return(predictions)

In [ ]:
# Test Backprop on Seeds dataset
seed(1)

# load and prepare data
filename = '/content/drive/MyDrive/Colab Notebooks/Development lab/seeds_dataset.csv'
dataset = load_csv(filename)

for i in range(len(dataset)):
  try:
	  dataset[i] = clean_string(dataset[i])
  except ValueError:
    print("line ->", i)

In [ ]:
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# set optimization parameters
n_folds = 5
l_rate = 0.5
n_epoch = 500
n_hidden = 5

# evaluate algorithm
scores = evaluate_algorithm(dataset, fit, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

>epoch=0, lrate=0.500, error=191.051, accurace=-19005.131
>epoch=1, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=2, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=3, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=4, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=5, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=6, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=7, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=8, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=9, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=10, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=11, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=12, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=13, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=14, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=15, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=16, lrate=0.500, error=168.000, accurace=-16700.000
>epoch=